# SSM 계산 : 차량 페어, 롱타입
* Heatmap으로 나타내기
* 시공도로 나타내기

# Import

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

import math # arctangent; math.atan 사용 목적
import statistics

from SSM_base import *
from SSM_time_based import *
from SSM_deceleration_based import *
from SSM_energy_based import *

import pyarrow.parquet as pq
#df = pq.read_pandas('data.parquet').to_pandas()

In [2]:
help(points)

Help on function points in module SSM_base:

points(local_x, local_y, velocity_x, velocity_y, v_len, v_wid)
    리뉴얼 :: 차량의 4개 꼭지점 좌표를 반환하는 함수
    각 차량 포인트 추출 : corner_1x, corner_1y, ... corner_4x, corner_4y



# Load Dataset

In [3]:
working_dir = 'D:/OneDrive/Projects/2023_SSM_Feasibility/Dataset'

In [4]:
file_list = ['LV_long_trajectories-0750am-0805am.parquet', 'LV_long_trajectories-0805am-0820am.parquet', 'LV_long_trajectories-0820am-0835am.parquet']

In [5]:
folder_name = '02_processed_long'
file_list = os.listdir(os.path.join(working_dir, '02_weave_temp'))

In [6]:
file_list[:3]

['1000_1001_weave.parquet',
 '1000_1005_weave.parquet',
 '1000_1006_weave.parquet']

# Calculation

## Settings
* 공통 상수 설정

In [7]:
threshold_TTC = 3 # time-based SSM 임계값
threshold_DRAC = 3.4
threshold_PET = 1

tmsec = 0.1 # 각 row별 시간단위(초)
g = 9.81 # 중력가속도
f = 0.35 # 마찰계수. friction coeefficient
delta_t = 1 # Reaction Delay time. 운전자 반응시간
MADR = 8.45 # 차량의 최대 감속 능력. 날씨에 따라, 차종에 따라 다르다. Saccomanno et al.(2008) 참조
deceleration_rate = 0.3 * g # 차량 감속도. 3.3도 됨

## Time-based SSM
* Temporal Proximal Indicator

In [8]:
# 꼭지점 4개 만들기
# 각 차량의 1,2,3,4번 꼭지점 좌표를 구한다.

warnings.filterwarnings('ignore')

for file in tqdm(file_list[:12871]):

    file_path = os.path.join(working_dir, '02_weave_temp', file)
    df = pq.read_pandas(file_path).to_pandas()
    
    df['new_local_x'], df['new_local_y'] = zip(*df.apply(lambda x: NGSIM_center(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
    df['new_LV_local_x'], df['new_LV_local_y'] = zip(*df.apply(lambda x: NGSIM_center(x.LV_local_x, x.LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))
    
    df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.apply(lambda x: points(x.new_local_x, x.new_local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
    df['LV_point_1'], df['LV_point_2'], df['LV_point_3'], df['LV_point_4'] = zip(*df.apply(lambda x: points(x.new_LV_local_x, x.new_LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))
    
    ## Nearest_point 추출
    df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))
    
    ## Shortest Distance 구하기 : ACT 용
    df['V_LV_point_delta'], df['V_LV_shortest_vector'], df['V_LV_shortest_distance'] = zip(*df.apply(lambda x: shortest_distance(
        x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
        x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
        x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))
    
    df['overlap'] = df.apply(lambda x: overlap(x.LV_type, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.new_local_x, x.new_local_y, x.new_LV_local_x, x.new_LV_local_y, x.V_len, x.LV_len), axis = 1)
    
    df['potential_conflict_type'], df['degX'] = zip(*df.apply(lambda x: potential_conflict_type(x.LV_type, x.overlap, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.new_local_x, x.new_local_y, x.new_LV_local_x, x.new_LV_local_y, x.V_len, x.V_wid, x.LV_len, x.LV_wid), axis = 1))
    
    # 2차원 SSM
    df['T2'] = df.apply(lambda x: T2(
        x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
        x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
        x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
        x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1)
    
    df['TTC'] = df.apply(lambda x: TTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    
    df['MTTC'] = df.apply(lambda x: MTTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['CI_MTTC'] = df.apply(lambda x: CI_MTTC(x.new_local_x, x.new_LV_local_x, x.velocity, x.LV_velocity, x.acc_x, x.LV_acc_x, x.MTTC), axis = 1)

    df['ACT'], df['total_rel'], df['d_delta'] = zip(*df.apply(lambda x: ACT(
    x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
    x.LV_velocity_x, x.LV_velocity_y, x.LV_velocity_x_before, x.LV_velocity_y_before,
    x.acc_x, x.acc_y, x.LV_acc_x, x.LV_acc_y, 
    x.V_LV_shortest_vector, x.V_LV_shortest_distance), axis = 1))
    
    df['pPET'], df['time_gap'] = zip(*df.apply(lambda x: pPET(
    x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
    x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1))
    
    df['PSD'] = df.apply(lambda x: PSD(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, f, g, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['DSS'] = df.apply(lambda x: DSS(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, g, f, delta_t, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    #df['TIDSS'] = df.apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)

    df['PICUD'] = df.apply(lambda x: PICUD(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.D_x, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    
    df['MTC'] = df.apply(lambda x: MTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.D_x, x.V_len, x.LV_len, x.velocity_x, x.LV_velocity, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    
    df['MMTC'] = df.apply(lambda x: MMTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, g, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)


    df['DRAC'] = df.apply(lambda x: DRAC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.D_x, x.velocity_x, x.LV_velocity_x, x.LV_len, x.V_len, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    #df['TIDRAC'] = df.apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)

    df['MDRAC'] = df.apply(lambda x: MDRAC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.TTC, delta_t, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['DCIA'] = df.apply(lambda x: DCIA(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, delta_t, x.D_x, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['unsafety'] = df.apply(lambda x: unsafety(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, deceleration_rate, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['TIT'] = df.apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
    df['TIT2'] = df.apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
    df['TIACT'] = df.apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)
    df['TITAdv'] = df.apply(lambda x: TIT(x.pPET, threshold_PET, tmsec), axis = 1)
    df['TITimeGap'] = df.apply(lambda x: TIT(x.time_gap, threshold_PET, tmsec), axis = 1)
    
    df['TIPICUD'] = df.apply(lambda x: TIDSS(x.PICUD, tmsec), axis = 1)
    df['TIDSS'] = df.apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
    
    df['TIDRAC'] = df.apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)
    df['TIMDRAC'] = df.apply(lambda x: TIDRAC(x.MDRAC, threshold_DRAC), axis = 1)
    df['TIDCIA'] = df.apply(lambda x: TIDRAC(x.DCIA, threshold_DRAC), axis = 1)
    
    #### SAVE ####
    save_folder = '03_SSM_weave_temp'
    save_name = file
    save_path = os.path.join(working_dir, save_folder, save_name)
    
    delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
              'LV_point_1', 'LV_point_2', 'LV_point_3', 'LV_point_4',
              'V_LV_point_num', 'V_LV_point_n', 'LV_point_n', 'V_LV_point_num2',
              'V_LV_point_n2',	'LV_point_n2',	'V_LV_point_delta',	'V_LV_shortest_vector',
              'V_LV_shortest_distance', 'total_rel', 'd_delta']
    
    df.drop(delete_cols, axis = 1, inplace = True)
    df.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 
    
print(f' ###################### ALL FILE IS COMPLETE! ######################')

100%|██████████████████████████████████████████████████████████████████████████| 12871/12871 [6:03:26<00:00,  1.69s/it]

 ###################### ALL FILE IS COMPLETE! ######################


In [9]:
# 통합파일을 저장할 것
save_folder = '03_SSM'
save_file = 'long_trajectories-0750am-0805am.parquet'
save_path = os.path.join(working_dir, save_folder, save_file)

# 통합할 파일 리스트
file_list = os.listdir(os.path.join(working_dir, '03_SSM_weave_temp'))
# 대표 스키마
schema_path = os.path.join(working_dir, '03_SSM_weave_temp', '1001_990_weave.parquet')
schema = pq.ParquetFile(schema_path).schema_arrow

# 통합파일로 저장하기
with pq.ParquetWriter(save_path, schema = schema) as writer:
    for file in tqdm(file_list):
        file_path = os.path.join(working_dir, '03_SSM_weave_temp', file)
        
        try:
            writer.write_table(pq.read_table(file_path, schema = schema))
        except:
            print(file)

100%|████████████████████████████████████████████████████████████████████████████| 44278/44278 [23:29<00:00, 31.42it/s]
